# Data pre-processing

In [1354]:
import pandas as pd

from MatchResult import MatchResult
from sklearn.preprocessing import OneHotEncoder

In [1355]:
match_cols = ['season', 'round'] + \
['date', 'time', 'referee', 'home_team', 'away_team', 'home_score', 'away_score'] + \
['home_coach'] + \
['home_player_' + str(i) for i in range(1, 12)] + \
['home_substitute_' + str(i) for i in range(1, 8)] + \
['away_coach'] + \
['away_player_' + str(i) for i in range(1, 12)] + \
['away_substitute_' + str(i) for i in range(1, 8)]

In [1356]:
train_raw = pd.read_csv('train-raw.csv')
train_raw.head()

,season,round,date,time,referee,home_team,away_team,home_score,away_score,home_coach,...,away_player_9,away_player_10,away_player_11,away_substitute_1,away_substitute_2,away_substitute_3,away_substitute_4,away_substitute_5,away_substitute_6,away_substitute_7
0,2005-06,1,28/08/2005,15:00,MASSIMO DE,ASCOLI,MILAN,1,1,Massimo Silva,...,Kaka,Andriy Shevchenko,Alberto Gilardino,Marek Jankulovski,Clarence Seedorf,Zeljko Kalac,Gennaro Gattuso,Manuel Rui Costa,Johann Vogel,Dario Simic
1,2005-06,1,27/08/2005,20:30,GIANLUCA PAPARESTA,FIORENTINA,SAMPDORIA,2,1,Cesare Prandelli,...,Lamberto Zauli,Francesco Flachi,Emiliano Bonazzoli,Marco Pisano,Vitaliy Kutuzov,Marco Borriello,Luca Castellazzi,Marco Zamboni,Simone Pavan,Gionata Mingozzi
2,2005-06,1,28/08/2005,15:00,TIZIANO PIERI,PARMA,PALERMO,1,1,Mario Beretta,...,Massimo Bonanni,Andrea Caracciolo,Stephen Makinwa,Nicola Santoni,Franco Brienza,Massimo Mutarelli,Giuseppe Biava,Michele Ferri,Mariano Gonzalez,Simone Pepe
3,2005-06,1,28/08/2005,15:00,PAOLO TAGLIAVENTO,INTER,TREVISO,3,0,Roberto Mancini,...,Reginaldo,Luigi Beghetto,Pinga,Roberto Chiappara,Dino Fava,Jehad Muntasser,Adriano Zancope,Francesco Parravicini,Anderson,Alberto Giuliatto
4,2005-06,1,27/08/2005,18:00,GIANLUCA ROCCHI,LIVORNO,LECCE,2,1,Roberto Donadoni,...,Alex Pinardi,Aleksei Eremenko,Graziano Pelle,Alfonso Camorani,Jaime Valdes,Giuseppe Cozzolino,Francesco Benussi,Marco Pecorari,Giuseppe Abruzzese,Davide Giorgino


In [1357]:
# test_raw = pd.read_csv('test-raw.csv')
# test_raw.head()

## Data visualization

Let's inspect our data a little bit more

In [1358]:
# todo

## Data manipulation
Now let's clean our raw data and add some additional features.

In [1359]:
df = pd.DataFrame(train_raw)
# df = df[:200]

In [1360]:
# convert date str to datetime
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
# sort by date column
df = df.sort_values(by='date')
df = df.reset_index(drop=True)
# round values to int
df['round'] = df['round'].astype(int)

### Additional features

#### Result column
Our model will try to predict match results, i.e. **home win**, **away win** or **draw**, so we need a result column to be used as our target.

In [1361]:
def get_match_result_from_score(home_score: int, away_score: int) -> MatchResult:
    if home_score == away_score:
        return MatchResult.draw
    if home_score > away_score:
        return MatchResult.home
    return MatchResult.away


def add_target_column(df: pd.DataFrame) -> pd.DataFrame:
    results = {'result': []}
    for index, row in df.iterrows():
        results['result'] += [get_match_result_from_score(row['home_score'], row['away_score']).name]
    df.insert(loc=df.columns.get_loc('home_score'), column='result', value=results['result'])
    return df

In [1362]:
# add target column
add_target_column(df)

,season,round,date,time,referee,home_team,away_team,result,home_score,away_score,...,away_player_9,away_player_10,away_player_11,away_substitute_1,away_substitute_2,away_substitute_3,away_substitute_4,away_substitute_5,away_substitute_6,away_substitute_7
0,2005-06,1,2005-08-27,20:30,GIANLUCA PAPARESTA,FIORENTINA,SAMPDORIA,home,2,1,...,Lamberto Zauli,Francesco Flachi,Emiliano Bonazzoli,Marco Pisano,Vitaliy Kutuzov,Marco Borriello,Luca Castellazzi,Marco Zamboni,Simone Pavan,Gionata Mingozzi
1,2005-06,1,2005-08-27,18:00,GIANLUCA ROCCHI,LIVORNO,LECCE,home,2,1,...,Alex Pinardi,Aleksei Eremenko,Graziano Pelle,Alfonso Camorani,Jaime Valdes,Giuseppe Cozzolino,Francesco Benussi,Marco Pecorari,Giuseppe Abruzzese,Davide Giorgino
2,2005-06,1,2005-08-28,15:00,MASSIMO DE,ASCOLI,MILAN,draw,1,1,...,Kaka,Andriy Shevchenko,Alberto Gilardino,Marek Jankulovski,Clarence Seedorf,Zeljko Kalac,Gennaro Gattuso,Manuel Rui Costa,Johann Vogel,Dario Simic
3,2005-06,1,2005-08-28,15:00,TIZIANO PIERI,PARMA,PALERMO,draw,1,1,...,Massimo Bonanni,Andrea Caracciolo,Stephen Makinwa,Nicola Santoni,Franco Brienza,Massimo Mutarelli,Giuseppe Biava,Michele Ferri,Mariano Gonzalez,Simone Pepe
4,2005-06,1,2005-08-28,15:00,PAOLO TAGLIAVENTO,INTER,TREVISO,home,3,0,...,Reginaldo,Luigi Beghetto,Pinga,Roberto Chiappara,Dino Fava,Jehad Muntasser,Adriano Zancope,Francesco Parravicini,Anderson,Alberto Giuliatto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,2008-09,22,2009-02-01,15:00,CHRISTIAN BRIGHI,ROBUR SIENA,LECCE,away,1,2,...,Guillermo Giacomazzi,Jose Castillo,Simone Tiribocchi,Andrea Ardito,Ndiaye Papa Waigo,Antonio Rosati,Raffaele Schiavi,Angelo,Daniele Cacia,Edinho
1355,2008-09,22,2009-02-01,15:00,PAOLO DONDARINI,ATALANTA,CATANIA,home,1,0,...,Jorge Martinez,Giuseppe Mascara,Michele Paolucci,Mariano Izco,Gionatha Spinesi,Takayuki Morimoto,Paolo Acerbis,Christian Silvestri,Cristian Llama,Pablo Ledesma
1356,2008-09,22,2009-02-01,15:00,GABRIELE GAVA,CHIEVOVERONA,SAMPDORIA,draw,1,1,...,Mirko Pieri,Giampaolo Pazzini,Antonio Cassano,Mattia Mustacchio,Claudio Bellucci,Antonio Mirante,Guido Marilungo,Pedro Obiang,Michele Ferri,Manuel Da Costa
1357,2008-09,22,2009-02-01,15:00,MATTEO SIMONE,GENOA,PALERMO,home,1,0,...,Mark Bresciano,Levan Mchedlidze,Edinson Cavani,Mirko Savini,Giulio Migliaccio,Roberto Guana,Samir Ujkani,Alberto Cossentino,Giovanni Tedesco,Davide Succi


### Rest days features
Rest days are very important for recovery.

In [1363]:
def count_days_between_dates(date1, date2) -> int:
    return (date1 - date2).dt.days

In [1364]:
# for i in range(5):
#     for home_or_away in HomeOrAway:
#         if i == 0:
#             df[f'{home_or_away.name}_team_rest_days'] = count_days_between_dates(df['date'], df[f'{home_or_away.name}_team_history_{i+1}_date'])
#         else:
#             df[f'{home_or_away.name}_team_history_{i}_rest_days'] = count_days_between_dates(df[f'{home_or_away.name}_team_history_{i}_date'], df[f'{home_or_away.name}_team_history_{i+1}_date'])

# todo: cannot count rest days for historical 5th games because we still miss the data about the 6th historical match

In [1365]:
# delete columns referring to the historical 6th matches
# df = df.loc[:, ~df.columns.str.contains('history_6')]

#### Datetime features
Add **year**, **month** and **day** features for all **date** value

In [1366]:
def get_exploded_datetime_values(df: pd.DataFrame) -> dict:
    data = {'year': [], 'month': [], 'day': [], 'hour': []}
    df['time'] = pd.to_datetime(df['time'], format="%H:%M")
    data['year'] += df['date'].map(lambda val: val.year).tolist()
    data['month'] += df['date'].map(lambda val: val.month).tolist()
    data['day'] += df['date'].map(lambda val: val.day).tolist()
    data['hour'] += df['time'].map(lambda val: val.hour).tolist()
    return data


def insert_exploded_datetime_values(df, exploded):
    df.insert(loc=df.columns.get_loc('time'), column='year', value=exploded['year'])
    df.insert(loc=df.columns.get_loc('time'), column='month', value=exploded['month'])
    df.insert(loc=df.columns.get_loc('time'), column='day', value=exploded['day'])
    df.insert(loc=df.columns.get_loc('time'), column='hour', value=exploded['hour'])
    return df


def explode_datetime_values(df: pd.DataFrame) -> pd.DataFrame:
    exploded = get_exploded_datetime_values(df)
    return insert_exploded_datetime_values(df, exploded)

In [1367]:
# explode datetime values
df = explode_datetime_values(df)
# drop date columns
df.drop('date', axis=1, inplace=True)
df.drop('time', axis=1, inplace=True)

The result of the pre-processing looks like this:

In [1368]:
df.head()

,season,round,year,month,day,hour,referee,home_team,away_team,result,...,away_player_9,away_player_10,away_player_11,away_substitute_1,away_substitute_2,away_substitute_3,away_substitute_4,away_substitute_5,away_substitute_6,away_substitute_7
0,2005-06,1,2005,8,27,20,GIANLUCA PAPARESTA,FIORENTINA,SAMPDORIA,home,...,Lamberto Zauli,Francesco Flachi,Emiliano Bonazzoli,Marco Pisano,Vitaliy Kutuzov,Marco Borriello,Luca Castellazzi,Marco Zamboni,Simone Pavan,Gionata Mingozzi
1,2005-06,1,2005,8,27,18,GIANLUCA ROCCHI,LIVORNO,LECCE,home,...,Alex Pinardi,Aleksei Eremenko,Graziano Pelle,Alfonso Camorani,Jaime Valdes,Giuseppe Cozzolino,Francesco Benussi,Marco Pecorari,Giuseppe Abruzzese,Davide Giorgino
2,2005-06,1,2005,8,28,15,MASSIMO DE,ASCOLI,MILAN,draw,...,Kaka,Andriy Shevchenko,Alberto Gilardino,Marek Jankulovski,Clarence Seedorf,Zeljko Kalac,Gennaro Gattuso,Manuel Rui Costa,Johann Vogel,Dario Simic
3,2005-06,1,2005,8,28,15,TIZIANO PIERI,PARMA,PALERMO,draw,...,Massimo Bonanni,Andrea Caracciolo,Stephen Makinwa,Nicola Santoni,Franco Brienza,Massimo Mutarelli,Giuseppe Biava,Michele Ferri,Mariano Gonzalez,Simone Pepe
4,2005-06,1,2005,8,28,15,PAOLO TAGLIAVENTO,INTER,TREVISO,home,...,Reginaldo,Luigi Beghetto,Pinga,Roberto Chiappara,Dino Fava,Jehad Muntasser,Adriano Zancope,Francesco Parravicini,Anderson,Alberto Giuliatto


In [1369]:
df = df.dropna()

# Data encoding

## Seasons

In [1370]:
season2index = {'20' + f'{i + 5}'.zfill(2) + '-' + f'{i + 6}'.zfill(2): i for i in range(16)}
df['season'] = df['season'].map(season2index)

C:\Users\david\AppData\Local\Temp\ipykernel_14796\1055702406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['season'] = df['season'].map(season2index)


## Players
We need to manually one-hot encode the names of the players because we have to consider all the player columns to construct a list containing all the unique values.

In [1371]:
def get_column_names_containing_str(df: pd.DataFrame, substring: str) -> list[str]:
    return df.loc[:, df.columns.str.contains(substring)].columns.values.tolist()

In [1372]:
# get all columns containing player names
player_columns = get_column_names_containing_str(df, 'player')
player_columns += get_column_names_containing_str(df, 'substitute')
home_player_columns = get_column_names_containing_str(df, 'home_player')
home_player_columns += get_column_names_containing_str(df, 'home_substitute')
away_player_columns = get_column_names_containing_str(df, 'away_player')
away_player_columns += get_column_names_containing_str(df, 'away_substitute')
all_unique_players = pd.concat([df[player_columns[i]] for i in range(len(player_columns))], axis=0).unique().tolist()

In [1373]:
result = []
for index, row in df.iterrows():
    home_encoding = [0] * len(all_unique_players)
    away_encoding = [0] * len(all_unique_players)
    for idx in range(len(home_player_columns)):
        home_player_idx = all_unique_players.index(row[home_player_columns[idx]])
        home_encoding[home_player_idx] = 1
        away_player_idx = all_unique_players.index(row[away_player_columns[idx]])
        away_encoding[away_player_idx] = 1
    result += [home_encoding + away_encoding]
home_away_player_columns = [f'home_player_{name}' for name in all_unique_players] + [f'away_player_{name}' for name in all_unique_players]
players_df = pd.DataFrame(result, columns=home_away_player_columns)
players_df.head()

,home_player_Sebastien Frey,home_player_Marco Amelia,home_player_Ferdinando Coppola,home_player_Cristiano Lupatelli,home_player_Julio Cesar,home_player_Christian Abbiati,home_player_Antonio Mirante,home_player_Morgan De Sanctis,home_player_Ivan Pelizzoli,home_player_Angelo Peruzzi,...,away_player_Mario DUrso,away_player_Federico Laurito,away_player_Ivan Fatic,away_player_Pape Diakhate,away_player_Oscar Torres,away_player_Manuel Mancini,away_player_Iago Falque,away_player_Daniele Ragatzu,away_player_Ciro Immobile,away_player_Edinho
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1374]:
df.shape

(1312, 50)

In [1375]:
df = df.drop(player_columns, axis=1)
df = pd.concat([df, players_df.set_index(df.index)], axis=1)
df.head()

,season,round,year,month,day,hour,referee,home_team,away_team,result,...,away_player_Mario DUrso,away_player_Federico Laurito,away_player_Ivan Fatic,away_player_Pape Diakhate,away_player_Oscar Torres,away_player_Manuel Mancini,away_player_Iago Falque,away_player_Daniele Ragatzu,away_player_Ciro Immobile,away_player_Edinho
0,0,1,2005,8,27,20,GIANLUCA PAPARESTA,FIORENTINA,SAMPDORIA,home,...,0,0,0,0,0,0,0,0,0,0
1,0,1,2005,8,27,18,GIANLUCA ROCCHI,LIVORNO,LECCE,home,...,0,0,0,0,0,0,0,0,0,0
2,0,1,2005,8,28,15,MASSIMO DE,ASCOLI,MILAN,draw,...,0,0,0,0,0,0,0,0,0,0
3,0,1,2005,8,28,15,TIZIANO PIERI,PARMA,PALERMO,draw,...,0,0,0,0,0,0,0,0,0,0
4,0,1,2005,8,28,15,PAOLO TAGLIAVENTO,INTER,TREVISO,home,...,0,0,0,0,0,0,0,0,0,0


In [1376]:
df.shape

(1312, 2364)

## Remaining features

In [1377]:
# one-hot encoding for all non-integer columns
df = pd.get_dummies(df)

In [1378]:
df

,season,round,year,month,day,hour,home_score,away_score,home_player_Sebastien Frey,home_player_Marco Amelia,...,away_coach_Roberto Mancini,away_coach_Rosario Pergolizzi,away_coach_Serse Cosmi,away_coach_Silvio Baldini,away_coach_Sinisa Mihajlovic,away_coach_Stefano Colantuono,away_coach_Stefano Pioli,away_coach_Walter Alfredo Novellino,away_coach_Walter Mazzarri,away_coach_Walter Zenga
0,0,1,2005,8,27,20,2,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0,1,2005,8,27,18,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,2005,8,28,15,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,2005,8,28,15,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,2005,8,28,15,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,3,22,2009,2,1,15,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1355,3,22,2009,2,1,15,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1356,3,22,2009,2,1,15,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1357,3,22,2009,2,1,15,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Data normalization

In [1379]:
# todo

In [1380]:
df.to_csv("train.csv", index=False)